# AE4441-16 CVRP


# VRP CLASS

In [ ]:
from vrp import VRP
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

### Random Dataset test

#### MTZ Formulation

In [ ]:
n = 5
k = 1
Q = 48

vrp = VRP()

vrp.setup_random_data(number_of_customers=n, number_of_vehicles=k, vehicle_capacity=Q,        
demand_lower=1, demand_higher=10)
vrp.subtour_type = 'DFJ'
vrp.setup()



In [ ]:
# vrp.model.setParam("MIPGap", 0.1)
# vrp.model.Params.Threads = 2

vrp.optimize()

# vrp.visualize()

### VISUALIZE

In [ ]:
import networkx as nx

In [ ]:
vrp.find_active_arcs()
tours = vrp.subtour(vrp.K, vrp.active_arcs)
print(tours)

In [ ]:

cmap = plt.cm.get_cmap('hsv', len(vrp.K)+1)

ax = plt.subplot()

for k in tours.keys():
    vehicle_color = cmap(k-1)[0:3]
    vehicle_arcs = tours[k]
    G = nx.DiGraph()
    for tour in vehicle_arcs:
        idx = 0
        for node in tour:
            node_pos = (vrp.nodes.iloc[node][0],vrp.nodes.iloc[node][1])
            G.add_node(node,pos=node_pos)

            if idx < len(tour)-1:
                node_i = tour[idx]
                node_j = tour[idx+1]
                edge_cost = round(vrp.c[(node_i,node_j)],2)
                G.add_edge( node_i, node_j, weight=edge_cost )
                idx += 1

        node_pos = nx.get_node_attributes(G,'pos')
        weights = nx.get_edge_attributes(G,'weight')

        # nx.draw_networkx_nodes(G, node_pos, ax=ax, node_size=600, node_color='w', edgecolors=vehicle_color)

        nx.draw(G,node_pos,ax=ax, node_color='w', edgecolors=vehicle_color, edge_color= vehicle_color, label='k='+str(k) )

        for node in node_pos.keys():
            pos = node_pos[node]   

            if node == vrp.V[0]:
                offset = -0.15
                comma_on = ','
            elif node == vrp.V[-1]:
                offset = 0.15
            else:
                offset = 0
                comma_on=''
            ax.text(pos[0] + offset, pos[1], s=str(node)+comma_on,horizontalalignment='center',verticalalignment='center')

        # nx.draw_networkx_edges(G,node_pos,ax=ax,edge_color=vehicle_color)

        # nx.draw_networkx_edge_labels(G,node_pos,ax=ax,edge_labels=weights)      

xmin = vrp.nodes.min()['x_coord'] - 1
ymin = vrp.nodes.min()['y_coord'] - 1

xmax = vrp.nodes.max()['x_coord'] + 1
ymax = vrp.nodes.max()['y_coord'] + 1

ax.set(xlim=(xmin, xmax), ylim=(ymin, ymax))


G = nx.DiGraph()
pos = (vrp.nodes.iloc[0][0],vrp.nodes.iloc[0][1])
G.add_node(0,pos=pos)
node_pos = nx.get_node_attributes(G,'pos')
nx.draw_networkx_nodes(G,node_pos,ax=ax, node_color='w', edgecolors='k')

plt.legend(loc='lower right')


plt.show()      

In [ ]:
G = nx.Graph()

In [ ]:
for node in vrp.V:
    node_pos = (vrp.nodes.iloc[node][0],vrp.nodes.iloc[node][1])
    G.add_node(node,pos=node_pos)

for edge in vrp.c.keys():
    if edge != (vrp.V[0],vrp.V[-1]) and edge[1] != vrp.V[-1]:
        G.add_edge(edge[0],edge[1],weight=round(vrp.c[edge],2))

In [ ]:
node_pos = nx.get_node_attributes(G,'pos')
weights = nx.get_edge_attributes(G,'weight')

In [ ]:
ax = plt.subplot()

offset = 0
nx.draw(G, node_pos, ax=ax, node_color='w', edgecolors='k')

for node in vrp.V:
    pos = (vrp.nodes.iloc[node][0],vrp.nodes.iloc[node][1])    

    if node == vrp.V[0]:
        offset = -0.05
        comma_on = ','
    elif node == vrp.V[-1]:
        offset = 0.05
    else:
        offset = 0
        comma_on=''
    
    plt.text(pos[0] + offset, pos[1], s=str(node)+comma_on, horizontalalignment='center', verticalalignment='center')

nx.draw_networkx_edge_labels(G,node_pos,edge_labels=weights)
plt.show()

#### DFJ formulation

In [ ]:
n = 5
k = 3

vrp = VRP()

vrp.setup_random_data(number_of_customers=n, number_of_vehicles=k,               
demand_lower=1, demand_higher=10)
vrp.subtour_type = 'DFJ'
vrp.setup()

vrp.model.setParam("MIPGap", 0.1)
vrp.model.Params.Threads = 2

vrp.optimize()

vrp.visualize()

### preset dataset

In [ ]:
import numpy as np
import pandas as pd

from vrp import VRP

In [ ]:
k = 5  # number of vehicles


In [ ]:
file_name="validation_data_A/A-n32-k5.vrp"

In [ ]:
f = open(file_name,'r')
lines = f.readlines()
f.close()

In [ ]:
found_nodes = False
found_demand = False

nodes = []
V = []
q = {}

depot_node = []

for line in lines:

    if "NAME" in line:
        datasetname = line.strip("NAME").replace(" : ","")

    elif "CAPACITY" in line:
        Q = int(line.strip("CAPACITY").replace(" : ",""))

    elif "NODE_COORD_SECTION" in line:
        found_nodes = True

    elif "DEMAND_SECTION" in line:
        found_nodes  = False
        found_demand = True
    elif "DEPOT_SECTION" in line:
        found_demand = False
    elif "EOF" in line:
        break

    elif found_nodes == True:
        node = line.split()

        node_no = int(node[0]) - 1
        node_x = int(node[1])
        node_y = int(node[2])

        V.append( node_no )
        nodes.append([node_x,node_y])

    elif found_demand == True:
        demand       = line.split()
        demand_node  = int(demand[0]) - 1
        demand_value = int(demand[1])

        if demand_node != 0:
            q[demand_node] = demand_value               

# add n+1 to all nodes
V.append(len(V))
# add destination depot node 
nodes.append(nodes[0])

V     = np.array(V)
nodes = np.array(nodes)

N = V[1:-1]


In [ ]:
vrp = VRP()

In [ ]:
vrp.n = len(N)
vrp.number_of_customers = vrp.n
vrp.K = np.arange(1,k+1)
vrp.V = V
vrp.N = N

vrp.nodes = pd.DataFrame(nodes,columns=['x_coord', 'y_coord'])
vrp.Q = Q
vrp.q = q
vrp.subtour_type = 'DFJ'

vrp.create_arcs()

In [ ]:
vrp.setup()
vrp.model.Params.Threads = 2


In [ ]:
vrp.optimize()

In [ ]:
vrp.visualize()

#### MTZ formulation

In [ ]:
vrp_mtz = VRP()

vrp_mtz.setup_preset_data(file_name="validation_data_A/A-n32-k5.vrp",
                          number_of_vehicles=k)


In [ ]:
vrp_mtz.visualize(plot_sol='n')

# PROTOTYPE

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product

In [ ]:
rnd = np.random
rnd.seed(420)

In [ ]:
n = 15  # number of clients
k = 2 # number of vehicles

xc = rnd.rand(n+1)*10
yc = rnd.rand(n+1)*10

In [ ]:
plt.plot(xc[0], yc[0], c='r', marker='s')
plt.scatter(xc[1:], yc[1:], c='b')

In [ ]:
N = np.arange(1,n+1)                            # set of clients
V = np.concatenate( ([0], N, [n+1]) )           # set of nodes (depot + client)
K = np.arange(1,k+1)                            # set of vehicles
A = [(i, j) for i in V for j in V if i != j]    # arcs between nodes and vehicle k

In [ ]:
node_vertices = []
for i in range(0,n+2):
    if i == 0:
        vertex = (xc[i],yc[i])
    elif i == n+1:
        vertex = (xc[0],yc[0])
    else:
        vertex = (xc[i],yc[i])

    node_vertices.append(vertex)

node_vertices = np.array(node_vertices)

In [ ]:
c = {(i, j):  ( (node_vertices[i,0]-node_vertices[j,0])**2 + (node_vertices[i,1]-node_vertices[j,1])**2 )**0.5 for i, j in A} # Calculate euclidian distance between each node

q = {i: np.random.randint(1,10) for i in N}        # amount that needs to be delivered

Q =  sum( q.values() )  // len(K)  + 1 # ensure that the demand can always be fulfilled

print("totaldemand", sum( q.values() ), "Totalcapacity",Q*len(K) )

## Model VRP

In [ ]:
from gurobipy import Model, GRB, quicksum, LinExpr, tuplelist

In [ ]:
mdl = Model()
mdl.Params.TimeLimit = 100  # seconds

In [ ]:
x = {}
for i,j in A:
    for k in K:
        x[i,j,k] = mdl.addVar(vtype=GRB.BINARY, name = "x_{}{}{}".format(i,j,k))
u = {}
for i in N:
    for k in K:
        u[i,k] = mdl.addVar(vtype=GRB.CONTINUOUS, name = "u_{}{}".format(i,k))

## *Objective function (minimization of transportation cost)*
$$min\sum_{k=1}^K \sum_{i=0}^{n} \sum_{j=1,j \neq i}^{n+1} c_{ij}x_{ij}^{k}$$

Where 
- Central depot and set of customers (nodes) $\{1,...,n\}$. Central depot is divided in origin (node $0$) and destination depot (node $n+1$)
- Fleet of vehicles $\{1,...,K\}$. $K$ number of vehicles
- Decision variable: $x_{ij}^{k} = 1$ if vehicle $k$ goes from node $i$ to node $j$

In [ ]:

obj = LinExpr()

for k in K:
    for i in V:
        if i < n+1:
            for j in V:
                if j >= 1 and j != i:
                    obj += c[i,j]*x[i,j,k]

mdl.setObjective(obj, sense=GRB.MINIMIZE)

# mdl.setObjective(quicksum(quicksum(c[i, j]*x[i,j,k] for i ,j in A) for k in K))

mdl.update()



## *Constraints*

Each vehicle must leave the depot
$$\sum_{j=1}^{n+1} x_{0j}^{k}, k = 1,..., K $$ 
Each vehicle must return to the depot
$$\sum_{j=0}^{n} x_{j,n+1}^{k}, k = 1,..., K $$

In [ ]:
# Each vehicle must leave the depot
for k in K:
    mdl.addConstr( quicksum(x[V[0],j,k] for j in V if j >= 1 ) == 1 , name=f"Start_{k}")

# Each vehicle must return the depot
for k in K:
    mdl.addConstr( quicksum(x[j,V[n+1],k] for j in V if j < n + 1 ) == 1 , name=f"Finish_{k}")

mdl.update()


Each customer must be visited by a vehicle: 

$$\sum_{k=1}^{K}\sum_{j=0,j \neq i}^{n} x_{ji}^{k}, i = 1,..., n $$ 


In [ ]:
# Each customer must be visited by a vehicle


for i in N:
    mdl.addConstr( quicksum( quicksum(x[j,i,k] for j in V if j < n +1 and j != i) for k in K) == 1 )

mdl.update()


If a vehicle visits a customer, then the same vehicle must leave that customer: 

$$\sum_{j=0,j \neq i}^{n} x_{ji}^{k} = \sum_{j=1,j \neq i}^{n+1} x_{ij}^{k} , i = 1,..., n, k = 1,...,K $$ 


In [ ]:

# If a vehicle visits a customer, then the same vehicle must leave that customer)
for i in N:
    for k in K:
        mdl.addConstr( lhs=quicksum(x[j,i,k] for j in V if j < n+1 and j!= i), sense=GRB.EQUAL, rhs= quicksum(x[i,j,k] for j in V if j >= 1 and j!= i))

mdl.update()


In [ ]:
subtour_type = 'DFJ'

Subtour elimination constraint (Dantzig-Fulkerson Johnson)

$$\sum_{i\in S}\sum_{j \in S,j \neq i} x_{ij} \leq |S| - 1$$

In [ ]:
def subtourelim(self, where=0):
    if where == GRB.callback.MIPSOL:

        active_arcs = []

        for i,j in A:
            for k in K:
                solutions = self.cbGetSolution(self._vars)
                if solutions[i,j,k] > 0.5:
                    active_arcs.append([i,j,k])

        active_arcs = np.vstack(active_arcs)

        tours = subtour(active_arcs)


        for k in tours.keys():
            if len(tours[k]) > 1:
                for tour in tours[k]:
                    S = np.unique(tour)
                    expr = quicksum(self._vars[i,j,k] for i in S for j in S if j != i)
                    self.cbLazy(expr <= len(S) - 1)
    else:
        active_arcs = []

        for i,j in A:
            for k in K:
                solutions = self._vars
                if solutions[i,j,k].x > 0.5:
                    active_arcs.append([i,j,k])

        active_arcs = np.vstack(active_arcs)

        tours = subtour(active_arcs)


        for k in tours.keys():
            if len(tours[k]) > 1:
                for tour in tours[k]:
                    S = np.unique(tour)
                    expr = quicksum(self._vars[i,j,k] for i in S for j in S if j != i)
                    mdl.addLConstr(expr <= len(S) - 1)


def subtour(active_arcs):
    tours = {}

    for k in K:
        vehicle_tours = []
        vehicle_arcs = active_arcs[np.where(active_arcs[:,2] == k)][:,0:2]
        start_node, finish_node = vehicle_arcs[0]
        # if finish_node == V[-1]:
        #     finish_node = V[0]

        tour = [start_node, finish_node]
        vehicle_arcs = np.delete(vehicle_arcs,[0],axis=0)

        subtour_done = 0

        while True:
            while True:
                next_node = np.where(vehicle_arcs[:,0] == finish_node)

                if next_node[0].size == 0:
                    vehicle_tours.append(tour)
                    break
                else:
                    start_node, finish_node = vehicle_arcs[next_node][0]
                    # if finish_node == V[-1]:
                    #     finish_node = V[0]
                    vehicle_arcs = np.delete(vehicle_arcs,next_node[0], axis=0)

                    tour.append(finish_node)

            if vehicle_arcs.size != 0:
                start_node, finish_node = vehicle_arcs[0]
                vehicle_arcs = np.delete(vehicle_arcs,[0], axis=0)

                # if finish_node == V[-1]:
                #     finish_node = V[0]

                tour = [start_node, finish_node]
            else:
                tours[k] = vehicle_tours
                break
        
    return tours


Subtour elimination constraint (miller-tucker-zemlin)

$$u_{j} - u_{i} \geq q_{j} - Q(1-x_{ijk}), i,j = \{1,....,n\}, i \neq j$$

In [ ]:
# Miller-Tucker-Zemlin formulation for subtour elimination

if subtour_type == 'MTZ':
    for k in K:
        for i,j in A:
            if i >= 1 and j >= 1:
                if i != n+1 and j != n+1:
                    mdl.addConstr( u[j,k] - u[i,k] >= q[j] - Q*(1 - x[i,j,k]) )

    # Capacity constraint
    for i in N:
        for k in K:
            mdl.addConstr(u[i,k] >= q[i])
            mdl.addConstr(u[i,k] <= Q)
            
# DFJ formulation for subtour elimination
elif subtour_type == 'DFJ':
    for k in K:
        mdl.addConstr( quicksum(quicksum(q[j]*x[i,j,k] for j in N if j !=i) for i in V if i < n + 1) <= Q )


mdl.update()

In [ ]:
mdl.write('test.lp')
mdl._vars = x

### OPTIMIZE

In [ ]:
if subtour_type == "MTZ":
    mdl.optimize()
elif subtour_type == "DFJ":
    while True:
        tour_length = 0
    # mdl.Params.lazyConstraints = 1
        mdl.optimize()
        subtourelim(mdl)
        active_arcs = []
        for i,j in A:
            for k in K:
                if x[i,j,k].x > 0.99:
                    active_arcs.append([i,j,k])

        active_arcs = np.vstack(active_arcs)
        tours       = subtour(active_arcs)
        for k in K:
            tour_length += len(tours[k])
        
        if tour_length == len(K):
            break
        
            



In [ ]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

In [ ]:
cmap = get_cmap(len(K)+1)

for k in K:
    vehicle_arcs = active_arcs[ np.where(active_arcs[:,2] == k) ]
    for i, j, k_ in vehicle_arcs:
        plt.plot([node_vertices[i,0], node_vertices[j,0]], [node_vertices[i,1], node_vertices[j,1]], c = cmap(k), zorder=0)

# plot depot
plt.plot(xc[0], yc[0], c='r', marker='s')
plt.annotate("$d_{}$".format(V[0]),(xc[0]+1,yc[0]+1) )

# plot customers
plt.scatter(xc[1:], yc[1:], c='b')
for i in N:
    plt.annotate("$q_{}$".format(i), (xc[i]+0.1,yc[i]-2.5) )